In [1]:
import os

import torch
from ultralytics import YOLO

print(torch.cuda.is_available())

idx = 3

exps = [
    "2-1 Our Aug+v8 Aug",
    "2-2 Our Aug+v8 Aug + Gussian",
    "2-3 Our Aug+v8 Aug+ Normalize",
    "2-4 Our Aug+v8 Aug + Gussian + Normalize",
]
model = YOLO(f'/home/raymond0920/yolov8_xiang/result epoch 6000/{exps[idx]}/weights/best.pt')

True

model loading mem before: 0G
model loading: 0:00:00.025753
model loading mem after: 0G 



In [2]:
results = model.predict([f"./3d_data_preprocess/{f}" for f in os.listdir("./3d_data_preprocess")], imgsz=1024, epochs=6000, patience=3000, workers=0, batch=2, pretrained=True, 
            hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=180, translate=0.5, scale=0, shear=0, perspective=0, flipud=0.5, fliplr=0.5, mosaic=1.0, mixup=0.5, copy_paste=0,
            conf=0.3, max_det=3000, single_cls=False, seed=0,half=True)

model to device:cuda:0 0:00:00.063479
model fuse 0:00:00.088307

0: 1024x1024 2 yellows, 2 reds, 2 blues, 1: 1024x1024 21 yellows, 31 reds, 51 blues, 2: 1024x1024 12 yellows, 35 reds, 69 blues, 3: 1024x1024 11 yellows, 35 reds, 64 blues, 4: 1024x1024 6 yellows, 34 reds, 77 blues, 5: 1024x1024 2 yellows, 5 reds, 14 blues, 6: 1024x1024 2 yellows, 3 reds, 8 blues, 7: 1024x1024 11 yellows, 14 reds, 41 blues, 8: 1024x1024 18 yellows, 29 reds, 71 blues, 9: 1024x1024 11 yellows, 17 reds, 38 blues, 10: 1024x1024 6 yellows, 3 reds, 22 blues, 11: 1024x1024 8 yellows, 22 reds, 67 blues, 12: 1024x1024 1 yellow, 2 reds, 4 blues, 13: 1024x1024 4 yellows, 19 reds, 43 blues, 14: 1024x1024 17 yellows, 27 reds, 54 blues, 15: 1024x1024 12 yellows, 15 reds, 36 blues, 16: 1024x1024 1 yellow, 22 reds, 52 blues, 17: 1024x1024 1 yellow, 11 reds, 32 blues, 18: 1024x1024 14 yellows, 30 reds, 77 blues, 19: 1024x1024 1 yellow, 9 reds, 19 blues, 20: 1024x1024 1 yellow, 2 reds, 9 blues, 21: 1024x1024 1 yellow, 6 re

In [3]:
from PIL import Image
import cv2
video = cv2.VideoWriter('./3d_predict/yolov8/video.mp4',cv2.VideoWriter.fourcc(*"XVID"),10,(720,720))

def get_num(path):
    return int(os.path.splitext(os.path.basename(path))[0])

for idx, r in enumerate(sorted(results, key=lambda r: get_num(r.path))):
    img_name = os.path.basename(r.path)
    img_n, img_t = os.path.splitext(img_name)
    
    r.orig_img = cv2.imread(f'./3d_data/{str(img_n).zfill(5)}.tiff')
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    video.write(im_array)
    im.save(f'./3d_predict/yolov8/{os.path.basename(r.path)}')  # save image
video.release()

OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [6]:
import os
with open("/home/raymond0920/yolov8_xiang/3d_test.txt", 'w') as f:
    f.writelines([f"/home/raymond0920/yolov8_xiang/3d_data_preprocess/{img}\n" for img in os.listdir("/home/raymond0920/yolov8_xiang/3d_data_preprocess")])

In [59]:
import random
import cv2
import math
from PIL import Image

def get_num(path):
    return int(os.path.splitext(os.path.basename(path))[0])

video = cv2.VideoWriter('./3d_predict/yolov7/video.mp4',cv2.VideoWriter.fourcc(*"XVID"),10,(720,720))
colors = [(0,255,255), (0,0,255), (255,0,0)]

def plot_one_box(x, img, color=None, label=None, line_thickness=1):
    # Plots one bounding box on image img
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)

for label in sorted(os.listdir("/home/raymond0920/yolov7_xiang/yolov7-main/runs/test/3d_test/labels"), key=lambda p:get_num(p)):
    idx = label.split('.')[0]
    img = cv2.imread(f'./3d_data/{idx.zfill(5)}.tiff')
    ih, iw, _ = img.shape
    
    with open(f"/home/raymond0920/yolov7_xiang/yolov7-main/runs/test/3d_test/labels/{label}") as f:
        labels = [line.split() for line in f.readlines()]
        labels = [(int(cls), ((float(x)-float(w)/2)*iw, (float(y)-float(h)/2)*ih, (float(x)+float(w)/2)*iw, (float(y)+float(h)/2)*ih), float(conf)) for (cls, x ,y ,w ,h, conf) in labels]
    for cls, box, conf in labels:
        if conf > 0.25:
            plot_one_box(box,img, color=colors[cls])

    video.write(img)
    im = Image.fromarray(img[..., ::-1])
    im.save(f'./3d_predict/yolov7/{idx}.jpg')
video.release()

OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
